In [ ]:
!pip install pyngrok
!pip install flask_ngrok
!pip install line-bot-sdk
!pip install flask

In [8]:
from tempfile import template
from flask import Flask, request, abort
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

def ngrok_start():#for colab
  ngrok.set_auth_token(userdata.get('auth_token'))
  ngrok.connect(5000)
  run_with_ngrok(app)
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    ImageMessage,
    VideoMessage,
    LocationMessage,
    TemplateMessage,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    PostbackAction
    )
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)
from linebot.v3.messaging import TemplateMessage, ConfirmTemplate, MessageAction

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('access_token'))
handler = WebhookHandler(userdata.get('Channel_secret'))


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):

  with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    action = event.message.text
    if action == "confirm":
          reply = TemplateMessage(
              alt_text="confirm template",
              template=ConfirmTemplate(
                text="你喜歡寶可夢嗎？",
                actions=[
                  MessageAction(label="是", text="是的"),
                  MessageAction(label="否", text="不，我喜歡神奇寶貝")]
                )
          )
    elif action == "carousel":
      carousel_template = CarouselTemplate(
      columns=[
        CarouselColumn(
            thumbnail_image_url='https://p2.bahamut.com.tw/B/2KU/90/9bb57f7476e40ecb85de48b7bc1j53m5.JPG',
            title='灌籃高手',
            text='熱血籃球漫畫',
            actions=[
                URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E7%81%8C%E7%AF%AE%E9%AB%98%E6%89%8B'),
                MessageAction(label='投票', text='我要投他')
            ]),
        CarouselColumn(
            thumbnail_image_url='https://img.myvideo.net.tw/images/MUS030/0000/7413/201810092234212023_2016.jpg',
            title='JOJO的奇妙冒險 黃金之風',
            text='熱血戰鬥漫畫',
            actions=[
                URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E9%BB%83%E9%87%91%E4%B9%8B%E9%A2%A8'),
                MessageAction(label='投票', text='我要投他')
            ])])

      reply =  TemplateMessage(alt_text='這是一個輪播模板',template=carousel_template)
    else:
      reply = TextMessage(text="Please type confirm")




    line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]))

ngrok_start()#for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9b63-34-145-162-91.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/May/2025 07:57:59] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 07:58:05] "POST /callback HTTP/1.1" 200 -
